In [ ]:
import numpy as np
import math
import random
from termcolor import colored
import copy

: 

In [ ]:
class Board:
    protected_cell = np.array([11, 22, 28, 39, 45, 56, 62, 73])
    protected_cell_2d = [ ( 2, 8), (2, 10) , (8, 2), (8, 16), (10,2 ), (10, 16), (16, 8) , (16, 10)]
    distance_to_intersect = 34
    final_distance = 7

    def __init__(self):
        self.player_pieces = np.array([[0, 0, 0, 0], [0, 0, 0, 0]])

    def __eq__(self, __value: object) -> bool:
        return np.array_equal(self.player_pieces, __value.player_pieces)

    def __ge__(self, __value: object) -> bool:
        """ check if self >= __value """
        # Assume x and y are your 2D arrays
        x = self.player_pieces
        y = __value.player_pieces

        # Define the two arrays
        eq = np.equal(x, y) 
        ge = np.greater_equal(x, y)

        # Find the index of the first False value in eq
        index = np.argmax(np.logical_not(eq))

        # Get the corresponding element from ge
        corresponding_element = ge.flatten()[index]

        return corresponding_element


    def is_second_player(self,player):
        return player==1
    
    def other_player(self,player):
        return (int)(player==0)
    
    
    def is_finished(self):
        if (np.all(self.player_pieces[0] == 84) or np.all(self.player_pieces[1] == 84)):
            return True
        return False

    
    def get_index(self, player, dist):
        """_summary_

        Args:
            player (int): which player has this piece
            dist (int): the value of piece, how many steps this piece has moved

        Returns:
            i,j: the index in the 2d array, usually using for printing
        """
        x=dist
        i,j=-1,-1
        if (x>=17 and x<=67):
            # detrmine quarter
            if(x//17==1):
                if (x<=24):
                    i,j=10,11+x-17
                elif (x==25):
                    i,j=9,18
                else:
                    i,j=8,33-x+11
            
            if(x//17==2):
                if (x<=41):
                    i,j=41-x,10
                elif (x==42):
                    i,j=0,9
                else:
                    i,j=x-43,8
            
            
            if(x//17==3):
                if (x<=58):
                    i,j=8,58-x
                elif (x==59):
                    i,j=9,0
                else:
                    i,j=10,x-60
                
        elif(1<=x<=8):
            i,j= 10+x, 9
        
        elif(9<=x<=16):
            i,j= 18-x+9, 10
        
                
        elif(68<=x<=75):
            i,j= 11+x-68, 8
        
        elif(76<=x<=83):
            i,j= 18-x+76, 9
        
        
        if (i==-1 or j==-1):
            return i,j
        
        if (self.is_second_player(player)):
            i,j=18-i,18-j
        
        return i,j
                    

    
    
    def print(self):
        """
        print the Board, Like:
            the empty cell: ⬜
            the protect cell: <⬜>
            the piece of the first player: red number (1,2,3,4)
            the piece of the second player: green number (1,2,3,4)
            if the piece on a protect cell print the number between < >
        """        
        mp={
            -1 : "   ",
            0 : " ⬜ ",
            1 : colored(" 1 ", "red"),
            2 : colored(" 2 ", "red"),
            3 : colored(" 3 ", "red"),
            4 : colored(" 4 ", "red"),
            5 : colored(" 1 ", "blue"),
            6 : colored(" 2 ", "blue"),
            7 : colored(" 3 ", "blue"),
            8 : colored(" 4 ", "blue"),
            
            
            10 : "<⬜>",
            11 : colored("<1>", "red"),
            12 : colored("<2>", "red"),
            13 : colored("<3>", "red"),
            14 : colored("<4>", "red"),
            15 : colored("<1>", "blue"),
            16 : colored("<2>", "blue"),
            17 : colored("<3>", "blue"),
            18 : colored("<4>", "blue"),
        }
        
        
        
        arr = np.zeros((19, 19))
        n = arr.shape[0]-1
        for i in range(8):
            for j in range(8):
                arr[i][j] = arr[i][n-j] = arr[n-i][j] = arr[n-i][n-j] = -1
        
        for i in range(3):
            for j in range(3):
                arr[8+i][8+j]  = -1
        
        
        
        for x in self.protected_cell_2d:
            arr[x[0]][x[1]]+=10

        
        for i in range(self.player_pieces.shape[0]):
          for j in range(self.player_pieces.shape[1]):
            
            x,y= self.get_index(i,self.player_pieces[i][j])
            if (x!=-1 and y!= -1):
                if(i==1 and arr[x][y]%10==0):
                    arr[x][y]+=4
                arr[x][y]+=1
            
        
        
        for i in range(n+1):
            for j in range(n+1):
                print (mp[arr[i][j]],end=" ")
            print()
            print()
        
    def is_win(self, player):
        if (np.all(self.player_pieces[player] == 84)):
            return True
        return False

    

    def evaluate(self):
        """
        we will make in the end
        calc the value of current Board, the value is between -100 and 100, like:
            100 mean  the first player certainly will win
            -100 mean  the second player certainly will win
            0 mean draw
            50 mean the first player likly to win
        """
        if (self.is_finished()):
            if (self.is_win(0)): 
                return 100
            return -100
        
        return random.randint(-99, 99)

        



    def can_move(self, player, piece, step):
        """
        Args:
            player (int): the player who is play (0,1)
            piece (int): the piece which will move (0,1,2,3)
            step (int): the step which will add to its position

        return:
            bool if it can move or not
        """
        dist = self.player_pieces[player][piece]
        
        # can't start (need Khal)
        if (dist==0 and step != 1):
            return False

        # can't end the game
        if (dist + step > 84):
            return False
        
        # opponnent in protected cell
        
        i,j = self.get_index(player, dist+step)
        
        if ((i,j) in self.protected_cell_2d):
            other = self.other_player(player)
            for x in self.player_pieces[other]:
                i2,j2 = self.get_index(other,x)
                if (i==i2 and j == j2):
                    return False
        
        return True
        
        

    def move(self, player, piece, step):
        """_summary_

         Args:
            player (int): the player who is play (0,1)
            piece (int): the piece which will move (0,1,2,3)
            step (int): the step which will add to its position

        return:
        it will check if we can move, if:
            we can't it will be return false, a copy of the same board
            we can it will return true, a copy of new board

        """
        if (not self.can_move(player,piece,step)):
            return copy.deepcopy(self)
        
        new_board = copy.deepcopy(self)
        
        dist = new_board.player_pieces[player][piece] = new_board.player_pieces[player][piece] + step
        
        
        i,j= new_board.get_index(player,dist)
        
        other = new_board.other_player(player)
        
        for ind,val in enumerate(new_board.player_pieces[other]):
            i2,j2 = new_board.get_index(other,val)
            if (i==i2 and j == j2):
                new_board.player_pieces[other][ind]=0
        
        
        return new_board
        
        
        
        
        

    def get_next_possible_move(self, player, steps):
        """_summary_

        Args:
            player (int): the player who is play (0,1)
            steps (array of int): an array which generate from throw the shell

        it will call rec_next_move and return a np array of boards
        """
        
        steps = np.sort(steps)
        return self.__rec_next_move(player,steps,0)

    def __rec_next_move(self, player, steps, step_index=0):
        """_summary_

        Args:
            player (int): the player who is play (0,1)
            steps (array of int): an array which generate from throw the shell
            index (int, optional): the index of step which we will move with it

        it will try to move with this step for each piece.
        then for each board generate call rec_next_move and increase index by 1
        then add the result of each recursion call to a list and return the list
        """
        if (step_index == len(steps)):
            return [self]
        
        ret = []
        for piece_index,val in enumerate(self.player_pieces[player]):
            new_board = copy.deepcopy(self)
                
            if (new_board.can_move(player,piece_index,steps[step_index])):
                
                new_board = new_board.move(player, piece_index,steps[step_index] )
                ret.extend( new_board.__rec_next_move(player,steps, step_index+1) )

        
        return ret
        
        

: 

In [ ]:
class Game:
    # the probability of head (1)
    p_head = 0.443
    max_depth = 2
    # max branching
    max_throw = 2
    # the min probability which is wcommand:cellOutput.enableScrolling?3262775c-2328-4622-bca6-87af4bd691e0e get move less than it we will cut the branch
    min_p = 1e-5
    shells_probabilities=np.array([0,0,0,0,0,0,0])
    shell_value={
        #  the steps, one step plus, repeate 
        0:(6,0,1),
        1:(10,1,1),
        2:(2,0,0),
        3:(3,0,0),
        4:(4,0,0),
        5:(24,1,1),
        6:(12,0,1)
    }
    def __init__(self):
        self.board = Board()
        Game.calc_shells_probabilities()
    
    @staticmethod
    def update_shells_probabilities(new_values):
       Game.shells_probabilities = np.array(new_values)
    
    
    @staticmethod
    def calc_shells_probabilities():        
        new_value=[]    
        for i in range(7):
            new_value.append(math.comb(6,i) * Game.p_head**i *(1-Game.p_head)**(6-i))
        Game.update_shells_probabilities(new_value)

    
    def throw_shells(self):
        """
        get a random weighted number where the weights is the shells_probabilities
        """
        numbers=np.array(np.nonzero(Game.shells_probabilities)).flatten()
        chosen_number = random.choices(numbers, Game.shells_probabilities, k=1)[0]
        return chosen_number
    
    
    def expect_minimax(self, board, player, depth, throw, curr_p):
        """ 
            we will code algorithims
        """


    def optimize(player, first, second):
        """_summary_

        Args:
            player (int): which player is optimize for, max (0) or min (1)
            first (board,value): the first board and its value
            second (board,value): the second board and its value
        
        return (board, value) which is optimal
        """
        if first[1]==-1000: return second
        if second[1]==-1000: return first
        if (player == 0):
            if first[1] > second[1]:
                return first
            return second
        else:
            if first[1] < second[1]:
                return first
            return second
        
        
    
    def minmax (self, board:Board, player, steps, curr_depth):
        """_summary_

        Args:
            board (Board): current board
            player (int): which player is optimize for, max (0) or min (1)
            steps (list of integer): the list of steps which player get by throw (or expected to get)
            curr_depth (int): number of turn which check it

        Returns:
            (Board, value): the best move and its value
        """
        
        if (curr_depth >=Game.max_depth or board.is_finished()):
            return (board, board.evaluate())
        
        
        other = self.board.other_player(player)
        next_boards = board.get_next_possible_move(player,steps)    
        optimal = (board, -1000)
        for i in next_boards:
            curr = self.minmax_expected_value(i, other, [], True, 1, 0, curr_depth+1)
            optimal = self.optimize(optimal, curr)

        return optimal
    
    
    def can_repeate(self, is_repeate, curr_probability, repeate_count):
        if (is_repeate and curr_probability >= Game.min_p and repeate_count <= Game.max_throw):
            return True
        return False
    
    def calc_expected_value(self, v):
        """_summary_

        Args:
            v (list of number): the value of each probability

        Returns:
            float: the expected value = sum(P[i]*v[i]) for i in [0, len(v)]
        """
        res=0
        for i in range(len(v)):
            res += Game.shells_probabilities[i] * v[i]
        
        return res 
    
    
    def minmax_expected_value(self, board, player, steps, is_repeate, curr_probability, repeate_count, curr_depth):
        """_summary_

        Args:
            board (Board): current board
            player (int): which player is optimize for, max (0) or min (1)
            steps (list of integer): the list of steps which player get by throw (or expected to get)
            is_repeate (bool): if has Khal
            curr_probability (float): Cumulative probability from the first throw of this player turn until now
            repeate_count (int): how much Khal get In a row
            curr_depth (int): number of turn which check it
        
        Returns:
            (Board, value): the best move and its value
        
        """
        
        if (curr_depth >=Game.max_depth or board.is_finished()):
            return (board, board.evaluate())
        
        if (self.can_repeate(is_repeate, curr_probability, repeate_count)):
            repeate_count +=1
            res= []
            for i in range(7):
                new_steps = copy.deepcopy(steps)
                curr_throw = Game.shell_value[i]
                new_steps.append(curr_throw[0])
                if(curr_throw[1]!=0):
                    new_steps.append(curr_throw[1])
                
                curr = self.minmax_expected_value(board,player,new_steps,curr_throw[2],curr_probability*Game.shells_probabilities[i],repeate_count+curr_throw[2],curr_depth)
                res.append(curr[1])
            
            ev = self.calc_expected_value(res)
            return (board, ev)
        
        
        return self.minmax(board,player,steps,curr_depth)
            
            
        
        
        
    
        
        
    def play(self):
        """
            the iterative for every player game 
        """
        while( not self.board.is_finished()):
            self.first_player_play()
            self.second_player_play()

    
    
    def first_player_play(self):
        repeate= True
        steps =[10,1]
        while repeate:
            val = self.throw_shells()
            repeate = (bool)(Game.shell_value[val][2])
            steps.append(Game.shell_value[val][0])
            if(Game.shell_value[val][1]!=0):
                steps.append(Game.shell_value[val][1])
            
        print (steps)
        
        
        board , value = self.minmax(self.board,0,steps,0)
        self.board=board
        board.print()
        
    
    
    
    def second_player_play(self):
       

        
        repeate= True
        steps =[]
        while repeate:
            inp = input("press enter to throw shells:")
            if (inp != ""):
              print("try agian, the input is false.")
            val = self.throw_shells()
            repeate = (bool)(Game.shell_value[val][2])
            print (f"you get {val} shell, so your repeate is {repeate}")
            steps.append(Game.shell_value[val][0])
            if(Game.shell_value[val][1]!=0):
                steps.append(Game.shell_value[val][1])
            
        
        while (len(steps)>0):    
            self.board.print()
            print ("you have this list to move use it:")
            print ("Steps:")
            for ind,val in enumerate(steps):
                print (f"the step number {ind} has value : {val}")
            
            print ("your pieces is:")
            for ind,val in enumerate(self.board.player_pieces[1]):
                print (f"the piece number {ind} has color #### , and is travle: {val}")
            
            next_steps = self.board.get_next_possible_move(1,np.array(steps))
            if(len(next_steps)==0):
                print ("you cannot move.")
                break
                
            
            print ("enter the number of piece then enter then the number of step.")

            piece = (int)(input("enter the number of piece you want to move"))
            step_index = (int)(input("enter the number of step you want to move by it"))
            
            if( step_index >= len(steps) or piece >=4 or  not self.board.can_move(1,piece,steps[step_index])):
                print("this move is not allowed")
            else:
                self.board = self.board.move(1,piece,steps[step_index])
                steps.pop(step_index)
            
            
            
        
            
        

    
    
    
  

: 

In [ ]:
g = Game()
g.play()

: 